In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import shapely
import numpy as np
from scipy.spatial import Delaunay
import torch

# Loading a shapefile

We downloaded a publicly available [shapefile](https://mis.bkg.bund.de/trefferanzeige?docuuid=D38F5B40-9209-4DC0-82BC-57FB575D29D7) and continued from there. Save this file and give its path to the next cell.

In [ ]:
path = "/home/shapefiles/Germany"

In [ ]:
df = gpd.read_file(path+"/NUTS250k/250_NUTS3.shp")
print(df.head())
domain = df.geometry.unary_union
domain

# Creation of Mesh

A mesh-file (.msh), readable for gmsh can be created with the following snippet. Gmsh can create a mesh using this .msh file. For more information we refer to the documentation of *deal.ii* and *gmsh*.

In [ ]:
print(path + "/" + "Germany" + "_coastline" + '.geo',)

In [ ]:
amount_points = len(border_b.exterior.coords)
with open(path + "/" + country_name + "_coastline" + '.geo', 'w') as f:
    f.write('// Gmsh project created on Tue Apr 11 16:14:17 2024')
    f.write('\n')
    f.write('SetFactory("OpenCASCADE");')
    f.write('\n')
    #add points
    for i, (x,y) in enumerate(border_b.exterior.coords):
        f.write(f'//+')
        f.write('\n')
        f.write(f'Point({i+1}) = {{ {x}, {y} , 1.0}};')
        f.write('\n')
    #add lines
    for i in range(amount_points -1): #last point = first point, self-lines are not allowed
        f.write("//+")
        f.write('\n')
        f.write(f'Line({i+1}) = {{{i+1}, {i+2}}};\n')
    #add curve loops to create the boundary
    curve_string = "1" #?? last bits by hand
    for i in range(2, amount_points):
        curve_string = curve_string + ", " + str(i)
    f.write("//+\n")
    f.write(f"Curve Loop(1) = {{ {curve_string} }};\n")
    f.write("//+\n")
    f.write("Surface(1) = {1};\n")
    # a
    # a
    f.write("//+\n")
    f.write("Mesh.Algorithm = 2;\n")
    f.write("Mesh.RecombineAll = 1;")
    f.write("Mesh.CharacteristicLengthFactor = .6;")
    f.write("Mesh.SubdivisionAlgorithm = 1;")
    f.write("Mesh.Smoothing = 20;")
    f.write("""Show "*";\n""")

# Creation of evaluation points and their adjacencies

In [ ]:
def Random_Points_in_Bounds(polygon, number):   
    minx, miny, maxx, maxy = polygon.bounds
    x = np.random.uniform( minx, maxx, number )
    y = np.random.uniform( miny, maxy, number )
    return x, y

In [ ]:
rectangle = shapely.geometry.Polygon([
    (3.3e6, 6.3e6),
    (3.3e6, 5.9e6),
    (3.9e6, 5.9e6),
    (3.9e6, 6.3e6)
])
coast = rectangle.difference(domain)
border_b = coast.buffer(8000).simplify(10000).buffer(-15000)
border_b

In [ ]:
x,y = Random_Points_in_Bounds(border_b,4000)
df = pd.DataFrame()
df['points'] = list(zip(x,y))
df['points'] = df['points'].apply(shapely.geometry.Point)
gdf_points = gpd.GeoDataFrame(df, geometry='points')

In [ ]:
gdf_points.plot()

In [ ]:
gdf_points[border_b.contains(gdf_points)].plot()

In [ ]:
gdf_output = gdf_points[border_b.contains(gdf_points)].dropna()
points = np.stack((gdf_output.points.x,gdf_output.points.y), axis=-1)
tri = Delaunay(points)

In [ ]:
plt.triplot(points[:,0], points[:,1], tri.simplices)
plt.plot(points[:,0], points[:,1], 'o')
plt.show()

In [ ]:
edges_set = set()
for simplex in tri.simplices:
    # Each simplex is a triangle with vertices a, b, c
    for i in range(3):
        # Extract edges by pairing vertices, and sort them
        # Sorting ensures that the edge (a, b) is the same as (b, a)
        edge = tuple(sorted([simplex[i], simplex[(i + 1) % 3]]))
        edges_set.add(edge)
edges = list(edges_set)

In [ ]:
edge_index_tensor = []
for e in edges:
    #print(e, points[e[0]], points[e[1]])
    x = points[e[0]]
    y = points[e[1]]
    dist = np.linalg.norm(x-y)
    #print(dist)
    line = shapely.geometry.LineString((x,y))
    if not border_b.exterior.intersection(line):
        #print(torch.tensor(e))
        #print(torch.tensor((torch.tensor(e)[1], torch.tensor(e)[0])))
        #print(torch.tensor(dist))
        edge_index_tensor.append(torch.cat((torch.tensor(e), torch.tensor(dist).unsqueeze(0))))
        reverse = torch.tensor((torch.tensor(e)[1], torch.tensor(e)[0]))
        edge_index_tensor.append(torch.cat((reverse, torch.tensor(dist).unsqueeze(0))))
#print(edge_index_tensor)
edge_index_tensor = torch.stack(edge_index_tensor)
print(edge_index_tensor)

In [ ]:
torch.save(edge_index_tensor, "germany_coastline_adjacency_increased")
np.savetxt("Germany_coastline_points_increased.csv", points, delimiter=",")